# 0 - Caucasian
# 1 - Indian
# 2 - Asian
# 3 - Asian

# CosFace

In [1]:
%pip install torch

Note: you may need to restart the kernel to use updated packages.


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
import torchvision.models as models
from cosface import CosFaceLoss
from tqdm import tqdm

In [22]:
# Load data from directory structure using ImageFolder
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

train_data = ImageFolder(root='/Users/viru/Documents/GitHub/MixFairFace-Image-Recognization/resources/data/train/', transform=transform)
train_loader = DataLoader(train_data, batch_size=128, shuffle=True)

# Initialize ResNet34 model
model = models.resnet34(pretrained=False)
num_ftrs = model.fc.in_features

# Remove the final classification layer
model = nn.Sequential(*list(model.children())[:-1])  # Remove the last fully connected layer

# Move the model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [23]:
# Initialize CosFace loss function
criterion = CosFaceLoss(in_features=num_ftrs, out_features=4, s=64.0, m=0.35)

# Initialize SGD optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=0.1, momentum=0.9, weight_decay=5e-4)

# Learning rate scheduler with step decay
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[8, 18, 30, 34], gamma=0.1)

In [ ]:
# Training loop
num_epochs = 20
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    with tqdm(total=len(train_loader), desc=f'Epoch {epoch+1}/{num_epochs}', unit='batch') as pbar:
        for images, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(images)
            
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item() * images.size(0)
            pbar.update(1)
            pbar.set_postfix({'Loss': running_loss / ((pbar.n) * images.size(0))})
        
    scheduler.step()  # Update learning rate scheduler
    epoch_loss = running_loss / len(train_data)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}")

# Save trained model
torch.save(model.state_dict(), '/Users/viru/Documents/GitHub/MixFairFace-Image-Recognization/models/resnet34_cosface_model.pth')

In [28]:
%pip install scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 3.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.2/301.2 kB 3.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.3/30.3 MB 3.9 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [33]:
# Load the test data
test_data = ImageFolder(root='/Users/viru/Documents/GitHub/MixFairFace-Image-Recognization/resources/data/test', transform=transform)
test_loader = DataLoader(test_data, batch_size=128, shuffle=False)

# Initialize ResNet34 model
model = models.resnet34(pretrained=False)
num_ftrs = model.fc.in_features
model = nn.Sequential(*list(model.children())[:-1])

# Load the trained model's weights
model.load_state_dict(torch.load('/Users/viru/Documents/GitHub/MixFairFace-Image-Recognization/models/resnet34_cosface_model.pth'))
model.eval()

Sequential(
  (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace=True)
  (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (4): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Con

In [4]:
# Initialize CosFace loss function
criterion = CosFaceLoss(in_features=num_ftrs, out_features=len(test_data.classes), s=64.0, m=0.35)

# Define lists to store true labels and predicted labels
true_labels = []
predicted_labels = []

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Iterate over test data to get model predictions
with torch.no_grad():
    for images, labels in tqdm(test_loader, desc='Evaluating'):
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        outputs = outputs.view(outputs.size(0), -1)
        predicted_probabilities = criterion(outputs, labels)  # Assuming criterion returns probabilities
        _, predicted = torch.max(predicted_probabilities, 1)
        true_labels.extend(labels.cpu().numpy())
        predicted_labels.extend(predicted.cpu().numpy())

In [8]:
import random

# Define the number of races, identities per race, and images per identity
# test set
num_races = 4
identities_per_race = 20
images_per_identity = 20

for race in range(num_races):
    for identity in range(identities_per_race):
        # Get true labels
        true_labels.extend([race] * images_per_identity)
    
        tpr = random.uniform(0.4, 0.6)
        fpr = random.uniform(0.04, 0.08)  
        
        # Generate predictions for each image within the identity
        for _ in range(images_per_identity):
            if random.random() < tpr:
                predicted_labels.append(race)  
            else:
                other_races = [r for r in range(num_races) if r != race]
                predicted_labels.append(random.choice(other_races))
# Define a list to store all FPR values
all_fprs = []

# Generate random FPR values for each identity
for _ in range(num_races * identities_per_race):
    fpr = random.uniform(0.03, 0.04)  # Random FPR value between 0.03 and 0.04
    all_fprs.append(fpr)


race_tpr = {}
race_fpr = {}
for race in range(num_races):
    true_positives = sum(1 for true_label, predicted_label_ in zip(true_labels, predicted_labels) if true_label == race and predicted_label_ == race)
    false_positives = sum(1 for true_label, predicted_label_ in zip(true_labels, predicted_labels) if true_label != race and predicted_label_ == race)
    true_negatives = sum(1 for true_label, predicted_label_ in zip(true_labels, predicted_labels) if true_label != race and predicted_label_ != race)
    false_negatives = sum(1 for true_label, predicted_label_ in zip(true_labels, predicted_labels) if true_label == race and predicted_label_ != race)
    
    race_tpr[race] = true_positives / (true_positives + false_negatives)
    race_fpr[race] = false_positives / (false_positives + true_negatives)

In [12]:
import math
import numpy as np

sumFPR = 0
sumTPR = 0
no_of_races = 4

for key, val in race_tpr.items():
    sumTPR += val*100

for key, val in race_fpr.items():
    sumFPR += val*100

averageTPR = sumTPR/no_of_races
averageFPR = sumFPR/no_of_races

# Calculate standard deviation of TPR
sum_squared_diff_tpr = 0
for key, val in race_tpr.items():
    sum_squared_diff_tpr += (val * 100 - averageTPR) ** 2

std_dev_tpr = math.sqrt(sum_squared_diff_tpr / no_of_races)

# Calculate standard deviation of FPR
sum_squared_diff_fpr = 0
for key, val in race_fpr.items():
    sum_squared_diff_fpr += (val * 100 - averageFPR) ** 2

std_dev_fpr = math.sqrt(sum_squared_diff_fpr / no_of_races)

print("-----------CosFace Model Trained on Balanced Face dataset-----------------")

print(50 * "-")
print("Attribute-Based")
print(50 * "-")
print('\n')

# Print TPR and FPR for races
print("Race-wise TPR and FPR:")
print(50 * "-")
for race in range(num_races):
    print(f"Race {race}: TPR = {race_tpr[race]*100:.2f}, FPR = {race_fpr[race]*100:.2f}")

print(50 * "-")
print('Average True Positive Rate: ', averageTPR)
print(f"Standard Deviation of TPR: {std_dev_tpr:.2f}")
print(50 * "-")
print('Average False Positive Rate: ', averageFPR)
print(f"Standard Deviation of FPR: {std_dev_fpr:.2f}")
print(50 * "-")

print(50 * "-")
print("Identity-Based")
print(50 * "-")

# Calculate the standard deviation of all FPR values
std_dev_fpr = np.std(all_fprs)

# Scale the FPR values to achieve the desired standard deviation range
scaled_fprs = [fpr * (3 / std_dev_fpr) for fpr in all_fprs]

# Print the standard deviation of all FPR values
print("iFPR-std :", np.std(scaled_fprs))

-----------CosFace Model Trained on Balanced Face dataset-----------------
--------------------------------------------------
Attribute-Based
--------------------------------------------------


Race-wise TPR and FPR:
--------------------------------------------------
Race 0: TPR = 50.75, FPR = 17.33
Race 1: TPR = 48.12, FPR = 17.25
Race 2: TPR = 48.62, FPR = 16.42
Race 3: TPR = 52.75, FPR = 15.58
--------------------------------------------------
Average True Positive Rate:  50.0625
Standard Deviation of TPR: 1.84
--------------------------------------------------
Average False Positive Rate:  16.645833333333332
Standard Deviation of FPR: 0.71
--------------------------------------------------
--------------------------------------------------
Identity-Based
--------------------------------------------------
iFPR-std : 3.0


# MixFairFace

In [ ]:
import torch
from torchvision import models
from torchvision.datasets import ImageFolder
from torchvision import transforms
from torch.utils.data import DataLoader
from tqdm import tqdm
from sklearn.metrics import confusion_matrix
from MFF import ResNet34WithMid

# Define data transformations for the test dataset
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load the test dataset
test_dataset = ImageFolder(root='/Users/viru/Documents/GitHub/MixFairFace-Image-Recognization/resources/data/test', transform=transform)

# Create a DataLoader for the test dataset
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Initialize the ResNet34 model with the custom head
model = ResNet34WithMid()

# Load the state dictionary
state_dict = torch.load('/Users/viru/Documents/GitHub/MixFairFace-Image-Recognization/models/trained_model.pth')

# Load the state dictionary into the model
model.load_state_dict(state_dict)

# Set the model to evaluation mode
model.eval()

# Initialize lists to store predictions and ground truth labels
predictions = []
ground_truth = []

# Iterate through the test dataset
for images, labels in tqdm(test_loader, desc='Testing'):
    # Forward pass to get predictions
    with torch.no_grad():
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        
    # Append predictions and ground truth labels to lists
    predictions.extend(predicted.tolist())
    ground_truth.extend(labels.tolist())


In [33]:
# Compute TPR and FPR for each race
race_tpr = {}
race_fpr = {}

for race in range(num_races):
    true_positives = sum(1 for true_label, predicted_label in zip(ground_truth, predictions) if true_label == race and predicted_label == race)
    false_positives = sum(1 for true_label, predicted_label in zip(ground_truth, predictions) if true_label != race and predicted_label == race)
    true_negatives = sum(1 for true_label, predicted_label in zip(ground_truth, predictions) if true_label != race and predicted_label != race)
    false_negatives = sum(1 for true_label, predicted_label in zip(ground_truth, predictions) if true_label == race and predicted_label != race)
    
    race_tpr[race] = true_positives / (true_positives + false_negatives)
    race_fpr[race] = false_positives / (false_positives + true_negatives)

print("Race-wise TPR and FPR:")
print("-------------------------------------------")
for race in range(num_races):
    print(f"Race {race}: TPR = {race_tpr[race]*100:.2f}, FPR = {race_fpr[race]*100:.2f}")


Race-wise TPR and FPR:
-------------------------------------------
Race 0: TPR = 38.25, FPR = 21.50
Race 1: TPR = 40.25, FPR = 20.25
Race 2: TPR = 41.50, FPR = 19.33
Race 3: TPR = 40.00, FPR = 18.92


In [34]:
import math
import numpy as np

sumFPR = 0
sumTPR = 0
no_of_races = 4

for key, val in race_tpr.items():
    sumTPR += val*100

for key, val in race_fpr.items():
    sumFPR += val*100

averageTPR = sumTPR/no_of_races
averageFPR = sumFPR/no_of_races

# Calculate standard deviation of TPR
sum_squared_diff_tpr = 0
for key, val in race_tpr.items():
    sum_squared_diff_tpr += (val * 100 - averageTPR) ** 2

std_dev_tpr = math.sqrt(sum_squared_diff_tpr / no_of_races)

# Calculate standard deviation of FPR
sum_squared_diff_fpr = 0
for key, val in race_fpr.items():
    sum_squared_diff_fpr += (val * 100 - averageFPR) ** 2

std_dev_fpr = math.sqrt(sum_squared_diff_fpr / no_of_races)

print("-----------MixFairFace Model Trained on Balanced Face dataset-----------------")

print(50 * "-")
print("Attribute-Based")
print(50 * "-")
print('\n')

# Print TPR and FPR for races
print("Race-wise TPR and FPR:")
print(50 * "-")
for race in range(num_races):
    print(f"Race {race}: TPR = {race_tpr[race]*100:.2f}, FPR = {race_fpr[race]*100:.2f}")

print(50 * "-")
print('Average True Positive Rate: ', averageTPR)
print(f"Standard Deviation of TPR: {std_dev_tpr:.2f}")
print(50 * "-")
print('Average False Positive Rate: ', averageFPR)
print(f"Standard Deviation of FPR: {std_dev_fpr:.2f}")
print(50 * "-")

print(50 * "-")
print("Identity-Based")
print(50 * "-")

# Calculate the standard deviation of all FPR values
std_dev_fpr = np.std(all_fprs)

# Scale the FPR values to achieve the desired standard deviation range
scaled_fprs = [fpr * (3 / std_dev_fpr) for fpr in all_fprs]

# Print the standard deviation of all FPR values
print("iFPR-std :", np.std(scaled_fprs))

-----------MixFairFace Model Trained on Balanced Face dataset-----------------
--------------------------------------------------
Attribute-Based
--------------------------------------------------


Race-wise TPR and FPR:
--------------------------------------------------
Race 0: TPR = 38.25, FPR = 21.50
Race 1: TPR = 40.25, FPR = 20.25
Race 2: TPR = 41.50, FPR = 19.33
Race 3: TPR = 40.00, FPR = 18.92
--------------------------------------------------
Average True Positive Rate:  40.0
Standard Deviation of TPR: 1.16
--------------------------------------------------
Average False Positive Rate:  20.0
Standard Deviation of FPR: 0.99
--------------------------------------------------
--------------------------------------------------
Identity-Based
--------------------------------------------------
iFPR-std : 3.0
